In [ ]:
from google.colab import drive
drive.mount('/gdrive')
!ln -s "/gdrive/My Drive/Doodle Recognition/Dataset" "/content/Dataset"

In [ ]:
# from google.colab import drive
# drive.flush_and_unmount()

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import glob
import os
import csv
import matplotlib.figure
from sklearn.model_selection import train_test_split
import pandas as pd

In [ ]:
import ast
all_files = glob.glob(os.path.join('/content/Dataset/SampleData','*.csv'))
# all_files = glob.glob(os.path.join('/content/','*.csv'))
all_files.sort()
all_files

In [ ]:
import cv2
from PIL import Image
im_size = 64
n_class = 3

# redefine
def draw_to_img(strokes, im_size = im_size):
    fig, ax = plt.subplots()                        # plot the drawing as we did above
    for x, y in strokes:
        ax.plot(x, -np.array(y), lw = 10)
    ax.axis('off')
    
    fig.canvas.draw()                               # update a figure that has been altered
    A = np.array(fig.canvas.renderer._renderer)     # converting them into array
    print(A.shape)
    plt.close('all')
    plt.clf()
    
    A = (cv2.resize(A, (im_size, im_size)))  # image resizing to uniform format
    A=A[:, :, :3]  
    image=Image.fromarray(A)  
    black = (0,0,0)
    white=(255,255,255)
    pixels = image.getdata()
    newPixels = []

# Compare each pixel 
    for pixel in pixels:
      if pixel != white:
        newPixels.append(black)
      else:
        newPixels.append(white)
    newImg = Image.new("RGB",image.size)
    newImg.putdata(newPixels)
    img = cv2.cvtColor(np.float32(newImg), cv2.COLOR_BGR2GRAY).reshape(64,64,1)
    return img

In [ ]:
import ast
from google.colab import files

n_samples = 3
    #print(a)
    #filename = dir_path + a + '.csv'
X_save = np.empty([0,64,64,1])
for file in all_files:
    df = pd.read_csv(file, usecols=['drawing', 'word','recognized'])  # import the data in chunks
    df=df[df['recognized'] == True][:n_samples].reset_index()
    df['drawing'] = df.drawing.map(ast.literal_eval)                          # convert strings into list
    X = df.drawing.values
    for i in range(0,n_samples):
        img=draw_to_img(X[i])
        img=img.reshape(1,64,64,1)
        X_save=np.concatenate((X_save, img), axis=0)
    np.save(df.word[0]+'.npy', X_save)    
    files.download(df.word[0]+'.npy')
# covert strokes into array